<a href="https://colab.research.google.com/github/PardhivA/Stellar_classification/blob/main/Stellar_classfication_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###                     STELLAR CLASSIFICATION







In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns

In [ ]:

df = pd.read_csv('/content/star_classification 2.csv')
df['class'] = pd.factorize(df['class'])[0]




**Checking variance of each column**


In [ ]:
from urllib.request import urlopen
from numpy import loadtxt
from numpy import unique
path = 'https://raw.githubusercontent.com/PardhivA/Stellar_classification/main/RemovedHeaders.csv'
# load the dataset
data = loadtxt(urlopen(path), delimiter=',')
# summarize the number of unique values in each column
for i in range(data.shape[1]):
	num = len(unique(data[:, i]))
	percentage = float(num) / data.shape[0] * 100
	print('%d, %d, %.1f%%' % (i, num, percentage))

0, 77275, 77.3%
1, 99999, 100.0%
2, 99999, 100.0%
3, 93748, 93.7%
4, 92651, 92.7%
5, 91901, 91.9%
6, 92019, 92.0%
7, 92007, 92.0%
8, 430, 0.4%
9, 1, 0.0%
10, 6, 0.0%
11, 856, 0.9%
12, 100000, 100.0%
13, 3, 0.0%
14, 99295, 99.3%
15, 6284, 6.3%
16, 2180, 2.2%
17, 1000, 1.0%


Dropping the columns whose variance is very low.

In [ ]:
from urllib.request import urlopen
from numpy import loadtxt
from numpy import unique
path = 'https://raw.githubusercontent.com/PardhivA/Stellar_classification/main/RemovedHeaders.csv'
# load the dataset
data = loadtxt(urlopen(path), delimiter=',')
count=0

# summarize the number of unique values in each column
for i in range(data.shape[1]):
	num = len(unique(data[:, i]))
	percentage = float(num) / data.shape[0] * 100
	if(percentage < 8 and i!=13): 
		df.drop([df.columns[i-count]], inplace = True,axis=1)
		count=count+1
		
df   

,obj_ID,alpha,delta,u,g,r,i,z,spec_obj_ID,class,redshift
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,6.543777e+18,0,0.634794
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,1.176014e+19,0,0.779136
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,5.152200e+18,0,0.644195
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,1.030107e+19,0,0.932346
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,6.891865e+18,0,0.116123
...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,1.055431e+19,0,0.000000
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,8.586351e+18,0,0.404895
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,3.112008e+18,0,0.143366
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,7.601080e+18,0,0.455040


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(["class"], axis=1)
y = df['class']
sc = StandardScaler()
X = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)


In [ ]:
clsf = RandomForestClassifier(n_jobs=2,max_depth = 21,min_samples_leaf = 3,min_samples_split = 4, n_estimators = 44, random_state=42)
clsf.fit(X_train,y_train)
preds = clsf.predict(X_test)
print('accuracy = ',accuracy_score(preds,y_test))

accuracy =  0.9794


In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(clsf, X_train, y_train, cv=3)  
# Constructing the confusion matrix.
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_train_pred)



array([[49856,   572,   134],
       [ 1119, 14981,     2],
       [   15,     0, 18321]])

We used random search and grid search for finding best hyperparameters for random **forest** classifier **bold text**

In [ ]:
clsf = RandomForestClassifier(n_jobs=2, random_state=42)
hyperparameters = {'max_features':[None, 'auto', 'sqrt', 'log2'],
                   'max_depth':[None, 1, 5, 10, 15, 21],
                   'min_samples_leaf': [1, 2, 3],
                   'min_samples_split': [2, 4, 10],
                   'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)],
                   'criterion': ['entropy']}
rf_random = RandomizedSearchCV(clsf, hyperparameters, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
rf_random.best_params_

In [ ]:
clsf = RandomForestClassifier(n_jobs=2, random_state=42)
from sklearn.model_selection import GridSearchCV
hyperparameters = {'max_features':[None],
                   'max_depth':[20,21,19],
                   'min_samples_leaf': [3,2],
                   'min_samples_split': [ 4,5,6],
                   'n_estimators': [43,44,45],
                   'criterion': ['entropy']}
rf_grid = GridSearchCV(clsf, hyperparameters, cv = 10, n_jobs = -1, verbose = 2)
rf_grid.fit(X_train, y_train)

In [ ]:
rf_grid.best_params_